In [1]:
# Module 1 - Create a Blockchain

# To be intalled:
# Flask==0.12.2 : pip install Flask==0.12.2
# Postman HTTP Client: https://www.getpostman.com/

In [2]:
# Importing the libraries
import datetime
import hashlib
import json
from flask import Flask , jsonify

In [3]:
# Part-1 Building a blockchain

class Blockchain:

    def __init__(self):
        self.chain = []
        self.create_block( proof = 1 , previous_hash = '0')

    def create_block(self, proof, previous_hash):

        block = { 'index'         : len(self.chain) + 1 ,
                  'timestamp'     : str(datetime.datetime.now()),
                  'proof'         : proof , 
                  'previous_hash' : previous_hash 
                }

        self.chain.append(block)

        return (block)

    def get_previous_block(self):
        return (self.chain[-1])

    def proof_of_work(self, previous_proof):

        new_proof = 1
        check_proof = False

        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:4] == '0000' :
                check_proof = True
            else:
                new_proof +=1

        return (new_proof)

    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return (hashlib.sha256(encoded_block).hexdigest())


    def is_chain_valid(self, chain):
        
        previous_block = chain[0]
        block_index = 1
       
        while block_index < len(chain):

            block = chain[block_index]
            if block['previous_hash'] != self.hash(previous_block):
                return False
            
            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
            
            if hash_operation[:4] != '0000' :
                return False

            previous_block = block
            block_index +=1

        return True           


In [4]:
# Part 2 - Mining our Blockchain

# Creating a Web App
app = Flask(__name__)
#app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

# Creating a Blockchain 
blockchain = Blockchain()

# Mining a new block
@app.route('/mine_block', methods = ['GET'] )
def mine_block():

    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['proof']
    previous_hash = blockchain.hash(previous_block)

    proof = blockchain.proof_of_work(previous_proof)
    
    block = blockchain.create_block(proof, previous_hash)

    response= { 'message'       : 'Congratulations, you just mined a block!' ,
                'index'         : block['index'] ,
                'timestamp'     : block['timestamp'] ,
                'proof'         : block['proof'] ,
                'previous_hash' : block['previous_hash'] 
               }

    return jsonify(response) , 200



# Getting the full Blockchain
@app.route('/get_chain' , methods = ['GET'])
def get_chain():

    response = { 'chain'  : blockchain.chain ,
                 'lenght' : len(blockchain.chain ) 
               } 

    return jsonify(response) , 200



# Checking if the Blockchain is valid
@app.route('/is_valid' , methods = ['GET'])
def is_valid():

    is_valid = blockchain.is_chain_valid(blockchain.chain)
    if is_valid:
         response = { 'message' : 'All good. The Blockchain is valid' } 
    else:
         response = { 'message' : 'Houston we have a problem. The Blockchain is not valid' } 

    return jsonify(response) , 200


In [5]:
# Running the app
app.run(host = '0.0.0.0' , port = 5000)

# http://127.0.0.1:5000/get_chain
# http://127.0.0.1:5000/mine_block

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.6:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Aug/2022 18:23:27] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 18:23:39] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 18:23:40] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 18:23:41] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 18:23:41] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 18:23:51] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [10/Aug/2022 18:24:32] "GET /is_valid HTTP/1.1" 200 -


: 